# Cross-Encoder

<br>

![ce](https://weaviate.io/assets/images/cross-encoder-61b340b8d1f6359bb7650da2c59be11c.png)

<br>

## Cos'è un cross-encoder?

Un [Cross-Encoder](https://arxiv.org/abs/1908.10084) è un classificatore di coppie di frasi. Richiede in input una coppia di testi e ne classifica la similarità mediante un indice tra 0 e 1. A differenza dei [Bi-Encoder](https://arxiv.org/abs/1908.10084) non calcola gli embeddings della frase.

## Differenza tra bi-encoder e cross-encoder

Un **Bi-Encoder** prende in input una frase alla volta e ne calcola l'embedding. Per capire la similarità tra frasi su questi embeddings va calcolata una misura di similarità come la [similarità del coseno](https://it.wikipedia.org/wiki/Coseno_di_similitudine) o il [prodotto scalare](https://it.wikipedia.org/wiki/Prodotto_scalare).

Un **Cross-Encoder** prende in input una coppia di frasi simultaneamente, non ne calcola gli embeddings, ma le classifica secondo un indice di similarità compreso tra 0 e 1.

<br>

![bicross](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/Bi_vs_Cross-Encoder.png)

<br>

> N.B. se si vuole applicare il prodotto scalare assicurarsi che gli embeddings siano normalizzati

## Training di un Cross-Encoder usando [SBERT](https://www.sbert.net/)

Di seguito un esempio della creazione di un Cross-Encoder per l'italiano con la libreria sentence-transformers. Come per i Bi-Encoder ci sono diversi metodi per farlo a seconda del dataset scelto, qui verrà usato sempre un dataset formato Semantic Textual Similarity (STS) benchmark.

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install datasets

### Dataset

A seguito dello scaricamento del [dataset da Huggingface Hub](https://huggingface.co/datasets/stsb_multi_mt/viewer/it/train) bisogna preparare le coppie di frasi per il modello.

Il processo è molto simile a quello del Bi-Encoder ma qui, nel dataset di train le coppie vanno inserite in entrambe le combinazione quindi:
* ([frase1, frase2], sim)
* ([frase2, frase1], sim)

perchè lo score deve essere simmetrico.
Per il dataset di valutazione e/o test non c'è bisogno.

> Per maggiori info sulle altre tecniche consultare gli [esempi](https://www.sbert.net/examples/training/cross-encoder/README.html#examples).

In [ ]:
from datasets import load_dataset
from sentence_transformers import InputExample
from torch.utils.data import DataLoader

In [ ]:
dataset_train = load_dataset("stsb_multi_mt", name="it", split="train")
dataset_test = load_dataset("stsb_multi_mt", name="it", split="test")

In [ ]:
gold_samples = []
batch_size = 16
for df in dataset_train:
    score = float(df['similarity_score'])/5.0 
    gold_samples.append(InputExample(texts=[df['sentence1'], df['sentence2']], label=score))
    gold_samples.append(InputExample(texts=[df['sentence2'], df['sentence1']], label=score))

train_dataloader = DataLoader(gold_samples, shuffle=True, batch_size=batch_size)

In [ ]:
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
import math
# We add an evaluator, which evaluates the performance during training
evaluator = CECorrelationEvaluator([ [x['sentence1'], x['sentence2']] for x in dataset_test], [x/5.0 for x in dataset_test['similarity_score']])

## Train del modello

Partendo sempre da un [BERT per l'italiano](https://huggingface.co/dbmdz/bert-base-italian-uncased) creiamo il nostro cross-encoder inizializzando l'head mediante il numero di etichette da predire.

> Nel nostro caso, modello STS abbiamo una solo etichetta, ovvero lo score. Se avessimo avuto un dataset formato *NLI ("contradiction", "entailment", "neutral")* avremmo inserito il numero delle etichette presenti.

Scelte il numero di epoche di addestramento e i warmup steps possiamo iniziare l'addestramento.

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

model_checkpoint = "dbmdz/bert-base-italian-uncased"
cross_encoder = CrossEncoder(model_checkpoint, num_labels=1)


In [ ]:
num_epochs = 4
evaluation_steps = 500

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [ ]:
cross_encoder.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=evaluation_steps,
          warmup_steps=warmup_steps,
          save_best_model=True,
        output_path = "cross-encoder-italian-bert-stsb/")

### Evaluation

Solitamente i Cross-Encoder hanno prestazioni migliori sull'inferenza rispetto ai Bi-Encoder, in questo caso non ci siamo focalizzati sulla metrica infatti il circa 81% di accuratezza non è altissimo per questo modello.

In [ ]:
evaluator(cross_encoder)

## A cosa servono i Cross-Encoder?

I cross-encoder possono essere usati ogni volta che bisogna classificare coppie di frasi, come detto hanno prestazioni migliori dei Bi-Encoder ma scalano male.

I Bi-Encoder a loro volta possono essere usati per tutti quei task come la ricerca, il clustering, ... comunque quando abbiamo tante frasi da confrontare.

Per dare una misura della poca scalabilità dei cross-encoder si pensi che fare clustering di 10000 frasi potrebbe richiedere il calcolo di 50 milioni combinazioni, circa 65 ore. Lo stesso task il bi-encoder lo risolve in pochi secondi.

### Retrieve & Re-Rank

<br>

![pipeline](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/InformationRetrieval.png)

<br>

I Cross-Encoder sono molto performanti se combinati con i Bi-Encoder, in particolare nella ricerca. Una particolare tecnica che li coinvolge è il [retrieve & re-rank](https://www.sbert.net/examples/applications/retrieve_rerank/README.html) che in poche parole consiste in:
* **retrieve**: applicare il Bi-Encoder su una vasta base di possibili risultati per fare una *"scrematura"*

<br>

![retrieve](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/SemanticSearch.png)

<br>

* **re-rank**: applicare il Cross-Encoder per ordinare un sottoinsieme di risultati e scartare quelli irrilevanti

Esempio, in un dataset di 1 milione di documenti viene applicato un bi-encoder che restituisce 100 risultati per la richiesta effettuata dall'utente, su questi 100 viene applicato il cross-encoder che restituirà i primi 10 in ordine di somiglianza.

<br>

![rerank](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/CrossEncoder.png)

<br>


> In SBERT si possono trovare diversi [cross-encoder pre-addestrati](https://www.sbert.net/docs/pretrained_cross-encoders.html)